In [ ]:
import os
import gc
import time
import json
import pickle
from pathlib import Path

import numpy as np
np.bool = np.bool_
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(0)

import lightgbm as lgb
from tqdm import tqdm

import mxnet as mx
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
mx.random.seed(0)

from model import *
from utils import *

import warnings

warnings.filterwarnings("ignore", message="WARNING:gluonts.model.forecast:The mean prediction is not stored in the forecast data; the median is being returned instead. This behaviour may change in the future.")

In [ ]:
def train_models(epochs, learning_rate):    
    save_dir = f'../result'
    os.makedirs(save_dir, exist_ok=True)
    
    for level_idx in range(11, 13):
        level_dir = os.path.join(save_dir, f'level {level_idx}')
        os.makedirs(level_dir, exist_ok=True)

        highlight_print(f"\n========== Level {level_idx} ==========")
        print(f"Loading dataset...", end=' ')
        dataset_start = time.time()
        with open(os.path.join('../dataset/else', f'dataset_level_{level_idx}.pkl'), 'rb') as f:
            dataset = pickle.load(f)
        print(f"{(time.time() - dataset_start)/60:.1f} minutes")

        print("Normalizing dataset...", end=' ')
        normalize_start = time.time()
        norm_train_dataset, norm_valid_dataset, norm_test_dataset, normalizer = normalize_dataset(
            dataset['train'], 
            dataset['valid'],
            dataset['test'],
        )
        with open(os.path.join(level_dir, 'normalizer.pkl'), 'wb') as f:
            pickle.dump(normalizer, f)
        print(f"{(time.time() - normalize_start)/60:.1f} minutes")

        estimators = create_estimators(
            level_idx=level_idx, 
            train_dataset=norm_train_dataset
        )
        for estimator_name, estimator in estimators.items():
            if estimator_name != 'WaveNet':
                continue
                
            estimator_dir = os.path.join(level_dir, estimator_name)
            if any(existing_dir.startswith(estimator_name) for existing_dir in os.listdir(level_dir)):
                continue
            os.makedirs(estimator_dir, exist_ok=True)

            estimator_start = time.time()
            highlight_print(f"\n---------- {estimator_name} ----------")
            print("Start training...")
            train_start = time.time()
            estimator.trainer = EarlyStoppingTrainer(
                epochs=epochs,
                learning_rate=learning_rate,
                num_batches_per_epoch=get_optimal_num_batches(mx.context.num_gpus()),
                patience=15,
            )
            predictor = estimator.train(
                training_data=norm_train_dataset,
                validation_data=norm_valid_dataset
            )
            predictor.serialize(Path(f"{level_dir}/{estimator_name}"))
            highlight_print(f"End training... {(time.time() - train_start)/60:.1f} minutes", color='green')

            print("Start predicting...")
            pred_start = time.time()
            train_forecasts_it, train_labels_it = make_evaluation_predictions(
                dataset=norm_train_dataset,
                predictor=predictor,
            )
            train_forecasts = list(train_forecasts_it)
            train_labels = list(train_labels_it)
            valid_forecasts_it, valid_labels_it = make_evaluation_predictions(
                dataset=norm_valid_dataset,
                predictor=predictor,
            )
            valid_forecasts = list(valid_forecasts_it)
            valid_labels = list(valid_labels_it)
            test_forecasts_it, test_labels_it = make_evaluation_predictions(
                dataset=norm_test_dataset,
                predictor=predictor,
            )
            test_forecasts = list(test_forecasts_it)
            test_labels = list(test_labels_it)
            highlight_print(f"End predicting... {(time.time() - pred_start)/60:.1f} minutes", color='green')

            print("Start saving...")
            pred_save_start = time.time()
            for i in range(len(train_forecasts)):
                train_forecasts[i] = normalizer.inverse_transform_forecast(train_forecasts[i])
            for i in range(len(valid_forecasts)):
                valid_forecasts[i] = normalizer.inverse_transform_forecast(valid_forecasts[i])
            for i in range(len(test_forecasts)):
                test_forecasts[i] = normalizer.inverse_transform_forecast(test_forecasts[i])
            for i in range(len(train_labels)):
                train_labels[i] = normalizer.inverse_transform_labels(train_labels[i])
            for i in range(len(valid_labels)):
                valid_labels[i] = normalizer.inverse_transform_labels(valid_labels[i])
            for i in range(len(test_labels)):
                test_labels[i] = normalizer.inverse_transform_labels(test_labels[i])
            with open(f"{level_dir}/{estimator_name}/train_labels.pkl", "wb") as f:
                pickle.dump(train_labels, f)
            with open(f"{level_dir}/{estimator_name}/train_forecasts.pkl", "wb") as f:
                pickle.dump(train_forecasts, f)
            with open(f"{level_dir}/{estimator_name}/valid_labels.pkl", "wb") as f:
                pickle.dump(valid_labels, f)
            with open(f"{level_dir}/{estimator_name}/valid_forecasts.pkl", "wb") as f:
                pickle.dump(valid_forecasts, f)
            with open(f"{level_dir}/{estimator_name}/test_labels.pkl", "wb") as f:
                pickle.dump(test_labels, f)
            with open(f"{level_dir}/{estimator_name}/test_forecasts.pkl", "wb") as f:
                pickle.dump(test_forecasts, f)
            highlight_print(f"End saving... {(time.time() - pred_save_start)/60:.1f} minutes", color='green')

            # print("Start plotting...")
            # plot_start = time.time()
            # for i in range(len(test_forecasts)):
            #     plt.figure(figsize=(12, 6))
            #     plt.plot(test_labels[i][-100:].to_timestamp(), label="Actual")
            #     plt.plot(pd.Series(test_forecasts[i].quantile(0.5), index=test_forecasts[i].start_date.to_timestamp() + pd.to_timedelta(range(len(test_forecasts[0].quantile(0.5))), unit='D')), label="Forecast")
            #     plt.title(f'{test_forecasts[i].item_id}')
            #     plt.xlabel('Date')
            #     plt.ylabel('Sales')
            #     plt.legend(loc="upper right")
            #     plt.savefig(os.path.join(estimator_dir, f'series_{i + 1}.png'))
            #     plt.close()
            # highlight_print(f"End plotting... {(time.time() - plot_start)/60:.1f} minutes", color='green')
                
            print("Start evaluating...")
            eval_start = time.time()
            evaluator = Evaluator(quantiles=(0.5,), ignore_invalid_values=True)
            train_metrics_all_id, train_metrics_per_id = evaluator(train_labels, train_forecasts)
            valid_metrics_all_id, valid_metrics_per_id = evaluator(valid_labels, valid_forecasts)
            test_metrics_all_id, test_metrics_per_id = evaluator(test_labels, test_forecasts)
            highlight_print(f"End evaluating... {(time.time() - eval_start)/60:.1f} minutes", color='green')

            print("Start saving...")
            eval_save_start = time.time()
            with open(f"{level_dir}/{estimator_name}/train_metrics_all_id.pkl", "wb") as f:
                pickle.dump(train_metrics_all_id, f)
            with open(f"{level_dir}/{estimator_name}/train_metrics_per_id.pkl", "wb") as f:
                pickle.dump(train_metrics_per_id, f)
            with open(f"{level_dir}/{estimator_name}/valid_metrics_all_id.pkl", "wb") as f:
                pickle.dump(valid_metrics_all_id, f)
            with open(f"{level_dir}/{estimator_name}/valid_metrics_per_id.pkl", "wb") as f:
                pickle.dump(valid_metrics_per_id, f)
            with open(f"{level_dir}/{estimator_name}/test_metrics_all_id.pkl", "wb") as f:
                pickle.dump(test_metrics_all_id, f)
            with open(f"{level_dir}/{estimator_name}/test_metrics_per_id.pkl", "wb") as f:
                pickle.dump(test_metrics_per_id, f)
            highlight_print(f"End saving... {(time.time() - eval_save_start)/60:.1f} minutes", color='green')

            highlight_print(f"\nTotal time: {(time.time() - estimator_start)/60:.1f} minutes", color='red')

            # rename estimator directory
            os.rename(estimator_dir, f"{estimator_dir}_{test_metrics_all_id['MSE']:.2f}")

            # reduce memory
            del estimator, predictor, train_forecasts, train_labels, test_forecasts, test_labels
            gc.collect()

train_models(epochs=500, learning_rate=1e-3)

In [ ]:
save_dir = '../result'
os.makedirs(save_dir, exist_ok=True)

for level_idx in range(1, 13):
    print(f"\n========== Level {level_idx} ==========")

    level_dir = os.path.join(save_dir, f'level {level_idx}')
    os.makedirs(level_dir, exist_ok=True)
    estimator_dir = os.path.join(level_dir, 'LightGBM')
    os.makedirs(estimator_dir, exist_ok=True)
    
    with open(os.path.join('../dataset/lgb', f'dataset_level_{level_idx}.pkl'), 'rb') as f:
        datasets = pickle.load(f)
    gc.collect()
    
    labels = {}
    preds = {}
    metrics = {}
    group_metrics = {}
    ids = datasets['train']['data']['id'].unique()
    for id in tqdm(ids):
        train_mask = datasets['train']['data']['id'] == id
        valid_mask = datasets['valid']['data']['id'] == id
        test_mask = datasets['test']['data']['id'] == id
        
        train_data = datasets['train']['data'][train_mask].drop(columns=['id'])
        valid_data = datasets['valid']['data'][valid_mask].drop(columns=['id'])
        test_data = datasets['test']['data'][test_mask].drop(columns=['id'])
        
        train_dataset = lgb.Dataset(
            train_data,
            label=datasets['train']['target'][train_mask].astype('float32'),
            free_raw_data=True
        )
        valid_dataset = lgb.Dataset(
            valid_data,
            label=datasets['valid']['target'][valid_mask].astype('float32'),
            reference=train_dataset,
            free_raw_data=True
        )
        
        model = lgb.train(
            {'objective': 'regression',
             'learning_rate': 0.001,
             'seed': 42},
            train_dataset,
            valid_sets=[train_dataset, valid_dataset],
            valid_names=['train', 'valid'],
        )

        test_label = datasets['test']['target'][test_mask]
        test_pred = model.predict(test_data)

        labels[id] = test_label
        preds[id] = test_pred.tolist()

        test_metrics = calculate_metrics(
            test_label,
            test_pred
        )
        metrics[id] = {k: float(v) for k, v in test_metrics.items()}
        
        if 'groups' in datasets['test']:
            series_groups = datasets['test']['groups'][test_mask].unique()
            for group in series_groups:
                if group not in group_metrics:
                    group_metrics[group] = {}

                group_mask = datasets['test']['groups'][test_mask] == group
                y_true_group = datasets['test']['target'][test_mask][group_mask]
                y_pred_group = test_pred[group_mask]

                group_metric = calculate_metrics(y_true_group, y_pred_group)
                group_metrics[group][id] = {
                    k: float(v) for k, v in group_metric.items()
                }
        
        del model, train_dataset, valid_dataset
        gc.collect()

    with open(os.path.join(estimator_dir, "labels.pkl"), "wb") as f:
        pickle.dump(labels, f)
    with open(os.path.join(estimator_dir, "predictions.pkl"), "wb") as f:
        pickle.dump(preds, f)
    with open(os.path.join(estimator_dir, "metrics_per_id.pkl"), "wb") as f:
        pickle.dump(metrics, f)
    with open(os.path.join(estimator_dir, "metrics_per_group.pkl"), "wb") as f:
        pickle.dump(group_metrics, f)
    
    test_mses = [metric['MSE'] for metric in metrics.values()]
    avg_mse = sum(test_mses) / len(test_mses)
    
    new_estimator_dir = f"{estimator_dir}_{avg_mse:.2f}"
    os.rename(estimator_dir, new_estimator_dir)
    
    del datasets, labels, preds, metrics, group_metrics, ids
    gc.collect()